In [44]:
import tensorflow.keras as keras
from ray.tune import track
import numpy as np
np.random.seed(0)

import tensorflow as tf
print(tf.__version__)
try:
    tf.get_logger().setLevel('INFO')
except Exception as exc:
    print(exc)
import warnings
warnings.simplefilter("ignore")

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM

from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import os
import ray
from ray import tune
import time
import inspect
import pandas as pd
import matplotlib.pyplot as plt
#plt.style.use('ggplot')
%matplotlib inline

'''
class TuneReporterCallback(keras.callbacks.Callback):
    """Tune Callback for Keras.
    
    The callback is invoked every epoch.
    """

    def __init__(self, logs={}):
        self.iteration = 0
        super(TuneReporterCallback, self).__init__()

    def on_epoch_end(self, batch, logs={}):
        self.iteration += 1
        track.log(keras_info=logs, mean_accuracy=logs.get("accuracy"), mean_loss=logs.get("loss"))
    
'''   
class TuneReporterCallback(keras.callbacks.Callback):
    """Tune Callback for Keras.
    
    The callback is invoked every epoch.
    """
    def __init__(self, logs={}):
        self.iteration = 0
        super(TuneReporterCallback, self).__init__()
    def on_epoch_end(self, batch, logs={}):
        self.iteration += 1
        if "acc" in logs:
            tune.report(keras_info=logs, val_loss=logs['val_loss'], mean_accuracy=logs["acc"], f1_m = logs['f1_m'], precision_m=logs['precision_m'], recall_m=logs['recall_m'])
        else:
            tune.report(keras_info=logs, val_loss=logs['val_loss'], mean_accuracy=logs.get("accuracy"), f1_m = logs.get('f1_m'), precision_m=logs.get('precision_m'), recall_m=logs.get('recall_m'))
        time.sleep(0.1)
        
from keras import backend as K
from sklearn.metrics import cohen_kappa_score
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))        
        
def create_model(learning_rate, RNN_units, dropout):
    assert learning_rate > 0 and RNN_units > 0 and dropout > 0, "Did you set the right configuration?"
    input_shape = (28, 28)
    model = Sequential()
    model.add(SimpleRNN(units=int(RNN_units), dropout=dropout, input_shape=input_shape,  activation='relu', name='RNN'))
    model.add(Dense(num_labels, activation = 'softmax', name = 'dense_output'))
    #optimizer = SGD(lr=learning_rate)
    optimizer = Adam(clipvalue=0.5, lr=learning_rate)
    model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy',f1_m,precision_m, recall_m])
    return model
        
def tune_mnist(config):  
    model = create_model(learning_rate=config['lr'], RNN_units=int(config['unit']), dropout=config['dropout'])  # TODO: Change me.
    checkpoint_callback = ModelCheckpoint(
        "model.h5", monitor='loss', save_best_only=True, save_freq=2)

    # Enable Tune to make intermediate decisions by using a Tune Callback hook. This is Keras specific.
    callbacks = [checkpoint_callback, TuneReporterCallback()]
    task_dataset = pickle.load(open('/root/Raytune_MNIST_RNN/task_dataset.pkl', "rb"))
    image_size = 28
    X_train = np.reshape(task_dataset[0],[-1, image_size, image_size]) 
    Y_train = task_dataset[1]
    X_test = np.reshape(task_dataset[2],[-1, image_size, image_size]) 
    Y_test = task_dataset[3]
    # Train the model
    hist = model.fit(
        X_train, Y_train, 
        validation_data=(X_test, Y_test),
        verbose=0, 
        batch_size=100, 
        epochs=10, 
        callbacks=callbacks)
    for key in hist.history:
        print(key)

# Random and uniform sampling for hypertune
def random_search(task_data, task_id=0, cpu_resouce_fed=4):
    import numpy as np; np.random.seed(5)  
    hyperparameter_space = {
        "lr": tune.loguniform(0.001, 0.1),  
        "unit": tune.uniform(20, 50),
        "dropout": tune.loguniform(0.01, 0.3),
    }  
    num_samples = 10 
    ####################################################################################################
    ################ This is just a validation function for tutorial purposes only. ####################
    HP_KEYS = ["lr", "unit", "dropout"]
    assert all(key in hyperparameter_space for key in HP_KEYS), (
        "The hyperparameter space is not fully designated. It must include all of {}".format(HP_KEYS))
    ######################################################################################################

    ray.shutdown()  # Restart Ray defensively in case the ray connection is lost. 
    ray.init(log_to_driver=False, num_cpus=cpu_resouce_fed)
    # We clean out the logs before running for a clean visualization later.
    ! rm -rf ~/ray_results/tune_mnist$task_id
    analysis = tune.run(
        tune_mnist, 
        name="Random_mnist_task"+str(task_id),
        verbose=1, 
        config=hyperparameter_space,
        num_samples=num_samples)
    time.sleep(1)

    assert len(analysis.trials) > 2, "Did you set the correct number of samples?"

    # Obtain the directory where the best model is saved.
    print("You can use any of the following columns to get the best model: \n{}.".format(
        [k for k in analysis.dataframe() if k.startswith("keras_info")]))
    print("=" * 10)
    logdir = analysis.get_best_logdir("keras_info/val_acc", mode="max")
    print('Best model:',analysis.get_best_trial(metric='keras_info/val_acc', mode='max'), 
          'lr:', analysis.get_best_config(metric='keras_info/val_acc', mode='max')['lr'], 'unit:', analysis.get_best_config(metric='keras_info/val_acc', mode='max')['unit'], 'dropout:', analysis.get_best_config(metric='keras_info/val_acc', mode='max')['dropout'] )
    # We saved the model as `model.h5` in the logdir of the trial.
    from tensorflow.keras.models import load_model
    tuned_model = load_model(logdir + "/model.h5", custom_objects =  {'f1_m': f1_m, 'precision_m': precision_m, 'recall_m': recall_m})
    tuned_model.summary()
    image_size = 28
    X_test = np.reshape(task_data[2],[-1, image_size, image_size])  
    Y_test = task_data[3]
    tuned_loss, tuned_accuracy, f1_score, precision, recall = tuned_model.evaluate(X_test, Y_test, verbose=0)
    print("Loss is {:0.4f}".format(tuned_loss))
    print("Tuned accuracy is {:0.4f}".format(tuned_accuracy))
    print ('F1-score = {0}'.format(f1_score))
    print ('Precision = {0}'.format(precision))
    print ('Recall = {0}'.format(recall))
    return(analysis.get_best_config(metric='keras_info/acc', mode='max'))

#PBT population based sampling 
def mutation_pbtsearch(task_data, task_id=0, cpu_resouce_fed=4):
    from ray.tune.schedulers import PopulationBasedTraining
    from ray.tune.utils import validate_save_restore
    scheduler = PopulationBasedTraining(
        time_attr="training_iteration",
        metric="mean_accuracy",
        mode="max",
        perturbation_interval=20,
        hyperparam_mutations={
            # distribution for resampling
            "lr": lambda: np.random.uniform(0.0001, 1),
            # allow perturbations within this set of categorical values
            "unit": [40, 60, 100], "dropout": [0.1, 0.2, 0.3], 
        }
    )

    old_dirs = os.listdir('/root/ray_results/')

    ray.shutdown()  # Restart Ray defensively in case the ray connection is lost. 
    ray.init(log_to_driver=False, num_cpus=cpu_resouce_fed)
    ! rm -rf ~/ray_results/PBT_mnist_task$task_id
    analysis = tune.run(
        tune_mnist,
        name="PBT_mnist_task"+str(task_id),
        scheduler=scheduler,
        reuse_actors=True,
        verbose=1,
        stop={
            "training_iteration": 100,
        },
        num_samples=10,

        # PBT starts by training many neural networks in parallel with random hyperparameters. 
        config={
        "lr": tune.loguniform(0.001, 0.1),  
        "unit": tune.uniform(20, 50),
        "dropout": tune.loguniform(0.01, 0.3),
    } )
    time.sleep(1)
    print("You can use any of the following columns to get the best model: \n{}.".format(
        [k for k in analysis.dataframe() if k.startswith("keras_info")]))
    print("=" * 10)
    logdir = analysis.get_best_logdir("keras_info/val_acc", mode="max")
    print('Best model:',analysis.get_best_trial(metric='keras_info/val_acc', mode='max'), 
          analysis.get_best_config(metric='keras_info/val_acc', mode='max'))
    # We saved the model as `model.h5` in the logdir of the trial.
    from tensorflow.keras.models import load_model
    tuned_model = load_model(logdir + "/model.h5", custom_objects =  {'f1_m': f1_m, 'precision_m': precision_m, 'recall_m': recall_m})
    tuned_model.summary()
    image_size = 28
    X_test = np.reshape(task_data[2],[-1, image_size, image_size])
    Y_test = task_data[3]
    tuned_loss, tuned_accuracy, f1_score, precision, recall = tuned_model.evaluate(X_test, Y_test, verbose=0)
    print("Loss is {:0.4f}".format(tuned_loss))
    print("Tuned accuracy is {:0.4f}".format(tuned_accuracy))
    print ('F1-score = {0}'.format(f1_score))
    print ('Precision = {0}'.format(precision))
    print ('Recall = {0}'.format(recall))
    return(analysis.get_best_config(metric='keras_info/acc', mode='max'))

#ASHA Schedular
def ASHA_search(task_data, task_id=0, cpu_resouce_fed=4):
    from ray.tune.schedulers import ASHAScheduler
    ray.shutdown()  # Restart Ray defensively in case the ray connection is lost. 
    ray.init(log_to_driver=False, num_cpus=cpu_resouce_fed)
    custom_scheduler = ASHAScheduler(
        metric='mean_accuracy',
        mode="max",
        reduction_factor = 4,
        grace_period=1)# TODO: Add a ASHA as custom scheduler here
    hyperparameter_space={
        "lr": tune.loguniform(0.001, 0.1),  
        "unit": tune.uniform(20, 50),
        "dropout": tune.loguniform(0.01, 0.3),
    } 
    ray.shutdown()  # Restart Ray defensively in case the ray connection is lost. 
    ray.init(log_to_driver=False, num_cpus=cpu_resouce_fed)
    ! rm -rf ~/ray_results/ASHA_mnist_task$task_id
    analysis = tune.run(
        tune_mnist, 
        scheduler=custom_scheduler, 
        config=hyperparameter_space, 
        verbose=1,
        num_samples=10,
        #resources_per_trial={"cpu":4},
        name="ASHA_mnist_task"+str(task_id)  # This is used to specify the logging directory.
    )
    time.sleep(1)
    print("You can use any of the following columns to get the best model: \n{}.".format(
        [k for k in analysis.dataframe() if k.startswith("keras_info")]))
    print("=" * 10)
    logdir = analysis.get_best_logdir("keras_info/acc", mode="max")
    print('Best model:',analysis.get_best_trial(metric='keras_info/acc', mode='max'), 
          analysis.get_best_config(metric='keras_info/acc', mode='max'))
    # We saved the model as `model.h5` in the logdir of the trial.
    from tensorflow.keras.models import load_model
    tuned_model = load_model(logdir + "/model.h5", custom_objects =  {'f1_m': f1_m, 'precision_m': precision_m, 'recall_m': recall_m})
    tuned_model.summary()
    image_size = 28
    X_test = np.reshape(task_data[2],[-1, image_size, image_size])
    Y_test = task_data[3]
    tuned_loss, tuned_accuracy, f1_score, precision, recall = tuned_model.evaluate(X_test, Y_test, verbose=0)
    print("Loss is {:0.4f}".format(tuned_loss))
    print("Tuned accuracy is {:0.4f}".format(tuned_accuracy))
    print ('F1-score = {0}'.format(f1_score))
    print ('Precision = {0}'.format(precision))
    print ('Recall = {0}'.format(recall))
    return(analysis.get_best_config(metric='keras_info/acc', mode='max'))


#HyperOpt Search 
def hyperopt_search(task_data, task_id=0, cpu_resouce_fed=4):
    from ray.tune.suggest import ConcurrencyLimiter
    from ray.tune.schedulers import AsyncHyperBandScheduler
    from ray.tune.suggest.hyperopt import HyperOptSearch

    
    search_space={
        "lr": tune.loguniform(0.001, 0.1),  
        "unit": tune.uniform(20, 50),
        "dropout": tune.loguniform(0.01, 0.3),
    }
    current_best_params = [{
    'lr': 0.01,
    'unit': 25,
    'dropout': 0.2,
    }]
    scheduler = AsyncHyperBandScheduler()
    
    algo = HyperOptSearch(points_to_evaluate=current_best_params)
    algo = ConcurrencyLimiter(algo, max_concurrent=cpu_resouce_fed)
    ray.shutdown()  # Restart Ray defensively in case the ray connection is lost. 
    ray.init(log_to_driver=False, num_cpus=cpu_resouce_fed)
    ! rm -rf ~/ray_results/BayesOptSearch_mnist_task$task_id
    analysis =tune.run(tune_Drebin,
        name="hyperopt_mnist_task"+str(task_id), verbose = 1,
        scheduler=scheduler,
        search_alg=algo,
        num_samples=10, 
        metric="mean_accuracy",
        mode="max",
        config=search_space,
        stop={"training_iteration": 150})
    time.sleep(1)
    #from ray.tune import Analysis as analysis
    #analysis = ray.tune.Analysis('/root/ray_results/BayesOptSearch_ASNM') 
    print("You can use any of the following columns to get the best model: \n{}.".format(
        [k for k in analysis.dataframe() if k.startswith("keras_info")]))
    print("=" * 10)
    logdir = analysis.get_best_logdir("keras_info/acc", mode="max")
    print('Best model:', analysis.get_best_config(metric='keras_info/acc', mode='max'))
    # We saved the model as `model.h5` in the logdir of the trial.
    from tensorflow.keras.models import load_model
    tuned_model = load_model(logdir + "/model.h5", custom_objects =  {'f1_m': f1_m, 'precision_m': precision_m, 'recall_m': recall_m})
    tuned_model.summary()
    image_size = 28
    X_test = np.reshape(task_data[2],[-1, image_size, image_size])
    Y_test = task_data[3]
    tuned_loss, tuned_accuracy, f1_score, precision, recall = tuned_model.evaluate(X_test, Y_test, verbose=0)
    print("Loss is {:0.4f}".format(tuned_loss))
    print("Tuned accuracy is {:0.4f}".format(tuned_accuracy))
    print ('F1-score = {0}'.format(f1_score))
    print ('Precision = {0}'.format(precision))
    print ('Recall = {0}'.format(recall))
    return(analysis.get_best_config(metric='keras_info/acc', mode='max'))

def BayesOptSearch(task_data, task_id=0, cpu_resouce_fed=4):
    from ray.tune.schedulers import AsyncHyperBandScheduler
    from ray.tune.suggest import ConcurrencyLimiter
    from ray.tune.suggest.bayesopt import BayesOptSearch
    
    search_space={
        "lr": tune.loguniform(0.001, 0.1),  
        "unit": tune.uniform(20, 50),
        "dropout": tune.loguniform(0.01, 0.3),
    } 
    scheduler = AsyncHyperBandScheduler()
    
    algo = ConcurrencyLimiter(BayesOptSearch(utility_kwargs={
        "kind": "ucb",
        "kappa": 2.5,
        "xi": 0.0
        }, metric = 'mean_accuracy', mode = 'max'), 
        max_concurrent=cpu_resouce_fed)
    ray.shutdown()  # Restart Ray defensively in case the ray connection is lost. 
    ray.init(log_to_driver=False, num_cpus=cpu_resouce_fed)
    ! rm -rf ~/ray_results/BayesOptSearch_mnist_task$task_id
    analysis =tune.run(tune_mnist,
        name="BayesOptSearch_mnist_task"+str(task_id), verbose = 1,
        scheduler=scheduler,
        search_alg=algo,
        num_samples=100, 
        metric="mean_accuracy",
        mode="max",
        config=search_space,
        stop={"training_iteration": 150})
    time.sleep(1)
    #from ray.tune import Analysis as analysis
    #analysis = ray.tune.Analysis('/root/ray_results/BayesOptSearch_ASNM') 
    print("You can use any of the following columns to get the best model: \n{}.".format(
        [k for k in analysis.dataframe() if k.startswith("keras_info")]))
    print("=" * 10)
    logdir = analysis.get_best_logdir("keras_info/acc", mode="max")
    print('Best model:', analysis.get_best_config(metric='keras_info/acc', mode='max'))
    # We saved the model as `model.h5` in the logdir of the trial.
    from tensorflow.keras.models import load_model
    tuned_model = load_model(logdir + "/model.h5", custom_objects =  {'f1_m': f1_m, 'precision_m': precision_m, 'recall_m': recall_m})
    tuned_model.summary()
    image_size = 28
    X_test = np.reshape(task_data[2],[-1, image_size, image_size])
    Y_test = task_data[3]
    tuned_loss, tuned_accuracy, f1_score, precision, recall = tuned_model.evaluate(X_test, Y_test, verbose=0)
    print("Loss is {:0.4f}".format(tuned_loss))
    print("Tuned accuracy is {:0.4f}".format(tuned_accuracy))
    print ('F1-score = {0}'.format(f1_score))
    print ('Precision = {0}'.format(precision))
    print ('Recall = {0}'.format(recall))
    return(analysis.get_best_config(metric='keras_info/acc', mode='max'))


def NeverGradSearch(task_data, task_id=0, cpu_resouce_fed=4):
    from ray.tune.suggest import ConcurrencyLimiter
    from ray.tune.schedulers import AsyncHyperBandScheduler
    from ray.tune.suggest.nevergrad import NevergradSearch
    import nevergrad as ng
    
    search_space={
        "lr": tune.loguniform(0.001, 0.1),  
        "unit": tune.uniform(20, 50),
        "dropout": tune.loguniform(0.01, 0.3),
    } 
    scheduler = AsyncHyperBandScheduler()
    
    algo = NevergradSearch(
        optimizer=ng.optimizers.OnePlusOne,
        # space=space,  # If you want to set the space manually
    )
    algo = ConcurrencyLimiter(algo, max_concurrent=cpu_resouce_fed)
    ray.shutdown()  # Restart Ray defensively in case the ray connection is lost. 
    ray.init(log_to_driver=False, num_cpus=cpu_resouce_fed)
    ! rm -rf ~/ray_results/BayesOptSearch_mnist_task$task_id
    analysis =tune.run(tune_mnist,
        name="NeverGradSearch_mnist_task"+str(task_id), verbose = 1,
        scheduler=scheduler,
        search_alg=algo,
        num_samples=10, 
        metric="mean_accuracy",
        mode="max",
        config=search_space,
        stop={"training_iteration": 150})
    time.sleep(1)
    #from ray.tune import Analysis as analysis
    #analysis = ray.tune.Analysis('/root/ray_results/BayesOptSearch_ASNM') 
    print("You can use any of the following columns to get the best model: \n{}.".format(
        [k for k in analysis.dataframe() if k.startswith("keras_info")]))
    print("=" * 10)
    logdir = analysis.get_best_logdir("keras_info/acc", mode="max")
    print('Best model:', analysis.get_best_config(metric='keras_info/acc', mode='max'))
    # We saved the model as `model.h5` in the logdir of the trial.
    from tensorflow.keras.models import load_model
    tuned_model = load_model(logdir + "/model.h5", custom_objects =  {'f1_m': f1_m, 'precision_m': precision_m, 'recall_m': recall_m})
    tuned_model.summary()
    image_size = 28
    X_test = np.reshape(task_data[2],[-1, image_size, image_size])
    Y_test = task_data[3]
    tuned_loss, tuned_accuracy, f1_score, precision, recall = tuned_model.evaluate(X_test, Y_test, verbose=0)
    print("Loss is {:0.4f}".format(tuned_loss))
    print("Tuned accuracy is {:0.4f}".format(tuned_accuracy))
    print ('F1-score = {0}'.format(f1_score))
    print ('Precision = {0}'.format(precision))
    print ('Recall = {0}'.format(recall))
    return(analysis.get_best_config(metric='keras_info/acc', mode='max'))

def OptunaSearch(task_data, task_id=0, cpu_resouce_fed=4):
    from ray.tune.suggest import ConcurrencyLimiter
    from ray.tune.schedulers import AsyncHyperBandScheduler
    from ray.tune.suggest.optuna import OptunaSearch
    
    search_space={
        "lr": tune.loguniform(0.001, 0.1),  
        "unit": tune.uniform(20, 50),
        "dropout": tune.loguniform(0.01, 0.3),
    } 
    scheduler = AsyncHyperBandScheduler()
    
    algo = OptunaSearch(metric="mean_accuracy",
        mode="max")
    algo = ConcurrencyLimiter(algo, max_concurrent=cpu_resouce_fed)
    ray.shutdown()  # Restart Ray defensively in case the ray connection is lost. 
    ray.init(log_to_driver=False, num_cpus=cpu_resouce_fed)
    ! rm -rf ~/ray_results/BayesOptSearch_mnist_task$task_id
    analysis =tune.run(tune_mnist,
        name="OptunaSearch_mnist_task"+str(task_id), verbose = 1,
        scheduler=scheduler,
        search_alg=algo,
        num_samples=10, 
        metric="mean_accuracy",
        mode="max",
        config=search_space,
        stop={"training_iteration": 150})
    time.sleep(1)
    #from ray.tune import Analysis as analysis
    #analysis = ray.tune.Analysis('/root/ray_results/BayesOptSearch_ASNM') 
    print("You can use any of the following columns to get the best model: \n{}.".format(
        [k for k in analysis.dataframe() if k.startswith("keras_info")]))
    print("=" * 10)
    logdir = analysis.get_best_logdir("keras_info/acc", mode="max")
    print('Best model:', analysis.get_best_config(metric='keras_info/acc', mode='max'))
    # We saved the model as `model.h5` in the logdir of the trial.
    from tensorflow.keras.models import load_model
    tuned_model = load_model(logdir + "/model.h5", custom_objects =  {'f1_m': f1_m, 'precision_m': precision_m, 'recall_m': recall_m})
    tuned_model.summary()
    image_size = 28
    X_test = np.reshape(task_data[2],[-1, image_size, image_size])
    Y_test = task_data[3]
    tuned_loss, tuned_accuracy, f1_score, precision, recall = tuned_model.evaluate(X_test, Y_test, verbose=0)
    print("Loss is {:0.4f}".format(tuned_loss))
    print("Tuned accuracy is {:0.4f}".format(tuned_accuracy))
    print ('F1-score = {0}'.format(f1_score))
    print ('Precision = {0}'.format(precision))
    print ('Recall = {0}'.format(recall))
    return(analysis.get_best_config(metric='keras_info/acc', mode='max'))

def ZOOptSearch(task_data, task_id=0, cpu_resouce_fed=4):
    from ray.tune.suggest.zoopt import ZOOptSearch
    from ray.tune.schedulers import AsyncHyperBandScheduler
    from zoopt import ValueType  # noqa: F401
    
    search_space={
        "lr": tune.loguniform(0.001, 0.1),  
        "unit": tune.uniform(20, 50),
        "dropout": tune.loguniform(0.01, 0.3),
    }
    scheduler = AsyncHyperBandScheduler()
    num_samples = 10
    zoopt_search_config = {
        "parallel_num": cpu_resouce_fed,
    }

    algo = ZOOptSearch(
        algo="Asracos",  # only support ASRacos currently
        budget=num_samples,
        # dim_dict=space,  # If you want to set the space yourself
        **zoopt_search_config)
    ray.shutdown()  # Restart Ray defensively in case the ray connection is lost. 
    ray.init(log_to_driver=False, num_cpus=cpu_resouce_fed)
    ! rm -rf ~/ray_results/BayesOptSearch_mnist_task$task_id
    analysis =tune.run(tune_mnist,
        name="ZOOptSearch_mnist_task"+str(task_id), verbose = 1,
        scheduler=scheduler,
        search_alg=algo,
        num_samples=num_samples, 
        metric="mean_accuracy",
        mode="max",
        config=search_space,
        stop={"training_iteration": 150})
    time.sleep(1)
    #from ray.tune import Analysis as analysis
    #analysis = ray.tune.Analysis('/root/ray_results/BayesOptSearch_ASNM') 
    print("You can use any of the following columns to get the best model: \n{}.".format(
        [k for k in analysis.dataframe() if k.startswith("keras_info")]))
    print("=" * 10)
    logdir = analysis.get_best_logdir("keras_info/acc", mode="max")
    print('Best model:', analysis.get_best_config(metric='keras_info/acc', mode='max'))
    # We saved the model as `model.h5` in the logdir of the trial.
    from tensorflow.keras.models import load_model
    tuned_model = load_model(logdir + "/model.h5", custom_objects =  {'f1_m': f1_m, 'precision_m': precision_m, 'recall_m': recall_m})
    tuned_model.summary()
    image_size = 28
    X_test = np.reshape(task_data[2],[-1, image_size, image_size])
    Y_test = task_data[3]
    tuned_loss, tuned_accuracy, f1_score, precision, recall = tuned_model.evaluate(X_test, Y_test, verbose=0)
    print("Loss is {:0.4f}".format(tuned_loss))
    print("Tuned accuracy is {:0.4f}".format(tuned_accuracy))
    print ('F1-score = {0}'.format(f1_score))
    print ('Precision = {0}'.format(precision))
    print ('Recall = {0}'.format(recall))
    return(analysis.get_best_config(metric='keras_info/acc', mode='max'))

1.14.0


In [115]:
import pickle
import pdb
import time
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        last_model_stats = Model_Perf_save
        for i,lay in enumerate(model.layers):
            last_model_size = last_model_stats['shape'][-1][2*i+1][0]
            layer_weights = lay.get_weights()
            layer_weights[0][:last_model_stats['weights'][-1][3*i].shape[0],:last_model_size] = last_model_stats['weights'][-1][3*i]
            if i != len(model.layers)-1: #for last (dense) layer
                layer_weights[1][:last_model_stats['weights'][-1][3*i+1].shape[0],:last_model_size] = last_model_stats['weights'][-1][3*i+1]
                layer_weights[2][:last_model_size] = last_model_stats['weights'][-1][3*i+2]
            else:
                layer_weights[1][:last_model_size] = last_model_stats['weights'][-1][3*i+1]
            model.layers[i].set_weights(layer_weights)      

    def on_batch_end(self, batch, logs={}):
        last_model_stats = Model_Perf_save
        for i,lay in enumerate(model.layers):
            last_model_size = last_model_stats['shape'][-1][2*i+1][0]
            layer_weights = lay.get_weights()
            layer_weights[0][:last_model_stats['weights'][-1][3*i].shape[0],:last_model_size] = last_model_stats['weights'][-1][3*i]
            if i != len(model.layers)-1: #for last (dense) layer
                layer_weights[1][:last_model_stats['weights'][-1][3*i+1].shape[0],:last_model_size] = last_model_stats['weights'][-1][3*i+1]
                layer_weights[2][:last_model_size] = last_model_stats['weights'][-1][3*i+2]
            else:
                layer_weights[1][:last_model_size] = last_model_stats['weights'][-1][3*i+1]
            model.layers[i].set_weights(layer_weights)  
        
def create_task(data_path):
    data = pickle.load(open(data_path, "rb"))
    return data

def measure_CPU_Mem():
    import psutil
    CPU_usage_dump = []
    Mem_usage_dump = []  
    while True:
        CPU_usage_dump.append(psutil.cpu_percent())
        Mem_usage_dump.append(psutil.virtual_memory().percent)
        f_cpu=open("CPU_used.txt", "wb")
        f_mem=open("Mem_used.txt", "wb")
        pickle.dump(CPU_usage_dump, f_cpu) 
        pickle.dump(Mem_usage_dump, f_mem)     
        f_cpu.close()
        f_mem.close()
        time.sleep(1)

import multiprocessing
task_list = create_task('mnist_tasks.pkl')
num_tasks=5
num_labels=10
Model_Perf_save = {}
Model_Perf_save['tr_acc'] = []
Model_Perf_save['val_acc'] = []
Model_Perf_save['precision'] = []
Model_Perf_save['recall'] = []
Model_Perf_save['f1_score'] = []
Model_Perf_save['shape'] = []
Model_Perf_save['weights'] = []
Model_Perf_save['learn_rate'] = []
cpu_resouce_fed = 3
for search_algo in [ 
                    random_search,
                    #ASHA_search,
                    #mutation_pbtsearch,
                    #BayesOptSearch,
                    #NeverGradSearch,
                    #OptunaSearch,
                    #ZOOptSearch,
                    #hyperopt_search
                    ]:
    cpu_mem_collection = multiprocessing.Process(target=measure_CPU_Mem)
    cpu_mem_collection.start()
    start_time = time.time()
    for task_id in range(0,num_tasks):
        !rm -rf task_dataset.pkl
        f = open('task_dataset.pkl', 'wb')
        pickle.dump(task_list[task_id], f)
        f.close()
        hyper_param = search_algo(task_list[task_id], task_id, cpu_resouce_fed)
        image_size = 28
        if task_id == 0:
            model = create_model(learning_rate=hyper_param['lr'], RNN_units=int(hyper_param['unit']), dropout=hyper_param['dropout'])
            #call one of the search algorithm
            history = model.fit(np.reshape(task_list[task_id][0],[-1, image_size, image_size]), 
                  task_list[task_id][1], batch_size=128, epochs=10, verbose=1,
                  validation_data=(np.reshape(task_list[task_id][2],[-1, image_size, image_size]), task_list[task_id][3]))
        else:
            rnn_units = hyper_param["unit"] + Model_Perf_save['shape'][-1][1][0]
            model = create_model(learning_rate=hyper_param['lr'], RNN_units=int(rnn_units), dropout=hyper_param['dropout'])
            history = model.fit(np.reshape(task_list[task_id][0],[-1, image_size, image_size]),
                  task_list[task_id][1], batch_size=128, epochs=10, verbose=1,
                  validation_data=(np.reshape(task_list[task_id][2],[-1, image_size, image_size]), task_list[task_id][3]), callbacks  = [LossHistory()])
        loss_and_metrics = model.evaluate(np.reshape(task_list[task_id][0],[-1, image_size, image_size]), task_list[task_id][1], verbose=0)
        Model_Perf_save['tr_acc'].append(loss_and_metrics[1])
        loss_and_metrics = model.evaluate(np.reshape(task_list[task_id][4],[-1, image_size, image_size]), task_list[task_id][5], verbose=0)
        Model_Perf_save['val_acc'].append(loss_and_metrics[1])
        Model_Perf_save['f1_score'].append(loss_and_metrics[2])
        Model_Perf_save['precision'].append(loss_and_metrics[3])
        Model_Perf_save['recall'].append(loss_and_metrics[4])
        Model_Perf_save['shape'].append([i.shape for i in model.get_weights()])
        Model_Perf_save['learn_rate'].append(hyper_param["lr"])
        Model_Perf_save['weights'].append(model.get_weights()) 
    end_time = time.time()
    print('Search algorithm {} took {}.'.format(search_algo.__name__, end_time - start_time))
    
    f=open("time_taken.txt", "a+")
    f.write('Time taken for algo {} is {}. \n'.format(search_algo.__name__, end_time-start_time))

    try:
        f_cpu=open("CPU_used.txt", "rb")
        f_mem=open("Mem_used.txt", "rb")
        cpu_usage = pickle.load(f_cpu)
        mem_usage = pickle.load(f_mem)
        f_cpu.close()
        f_mem.close()
        cpu_mem_collection.terminate()
        cpu_mem_collection.join()
        f.write('CPU used is {}. \n'.format(np.mean(cpu_usage)))
        f.write('Memory used is {}. \n \n'.format(np.mean(mem_usage)))
        f.close()
    except:
        pass

Trial name,status,loc,dropout,lr,unit,acc,iter,total time (s),val_loss,f1_m,precision_m
tune_mnist_070a6_00000,TERMINATED,,0.0201999,0.00277963,46.122,0.987427,10,12.7533,0.0570034,0.987479,0.98752
tune_mnist_070a6_00001,TERMINATED,,0.0800977,0.0687419,34.6523,0.865001,10,11.0569,0.246032,0.865104,0.865104
tune_mnist_070a6_00002,TERMINATED,,0.0274417,0.0340264,35.5525,0.95194,10,10.9895,0.13159,0.951921,0.952318
tune_mnist_070a6_00003,TERMINATED,,0.123244,0.00237379,22.4222,0.978741,10,10.7936,0.0587078,0.97874,0.97874
tune_mnist_070a6_00004,TERMINATED,,0.199422,0.00763165,24.7493,0.9775,10,10.3745,0.0525128,0.97749,0.97749
tune_mnist_070a6_00005,TERMINATED,,0.0273745,0.00353324,32.4271,0.982794,10,11.4658,0.0400779,0.9828,0.9828
tune_mnist_070a6_00006,TERMINATED,,0.0769428,0.0180957,37.3951,0.972537,10,12.0026,0.0643474,0.972498,0.972621
tune_mnist_070a6_00007,TERMINATED,,0.0236908,0.00340125,28.5406,0.984118,10,10.9394,0.037716,0.984122,0.984122
tune_mnist_070a6_00008,TERMINATED,,0.0175643,0.00451989,24.3249,0.980065,10,10.52,0.048732,0.980091,0.980214
tune_mnist_070a6_00009,TERMINATED,,0.0189807,0.0846956,48.8068,0.571677,10,11.2805,0.583765,0.56968,0.571962


2022-01-06 10:41:09,979	INFO tune.py:439 -- Total run time: 57.33 seconds (57.22 seconds for the tuning loop).


You can use any of the following columns to get the best model: 
['keras_info/loss', 'keras_info/acc', 'keras_info/f1_m', 'keras_info/precision_m', 'keras_info/recall_m', 'keras_info/val_loss', 'keras_info/val_acc', 'keras_info/val_f1_m', 'keras_info/val_precision_m', 'keras_info/val_recall_m'].
Best model: tune_mnist_070a6_00007 lr: 0.0034012474960852393 unit: 28.540576419240914 dropout: 0.023690843765683638
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
RNN (SimpleRNN)              (None, 28)                1596      
_________________________________________________________________
dense_output (Dense)         (None, 10)                290       
Total params: 1,886
Trainable params: 1,886
Non-trainable params: 0
_________________________________________________________________
Loss is 0.0385
Tuned accuracy is 0.9878
F1-score = 0.9876803159713745
Precision = 0.9876803159713745
Re

In [116]:
for j,search_algo in enumerate([
                    #ASHA_search, 
                    random_search,
                    #mutation_pbtsearch,
                    #BayesOptSearch,
                    #NeverGradSearch,
                    #OptunaSearch,
                    #ZOOptSearch,
                    #hyperopt_search
                    ]):
    print('Search Algorithm: {0}'.format(search_algo.__name__))
    print('Training accuracy for tasks:',Model_Perf_save['tr_acc'][j*num_tasks:j*num_tasks+num_tasks])
    print('Validation accuracy for tasks:',Model_Perf_save['val_acc'][j*num_tasks:j*num_tasks+num_tasks])
    print('Precision:',Model_Perf_save['precision'][j*num_tasks:j*num_tasks+num_tasks])
    print('Recall:',Model_Perf_save['recall'][j*num_tasks:j*num_tasks+num_tasks])
    print('F1 score:',Model_Perf_save['f1_score'][j*num_tasks:j*num_tasks+num_tasks])
    print('Nodes in hidden layer: ',[(i[0])for i in Model_Perf_save['shape']][j*num_tasks:j*num_tasks+num_tasks])
    print('Learning rates: ', Model_Perf_save['learn_rate'][j*num_tasks:j*num_tasks+num_tasks])
    print('------------------------')

Search Algorithm: random_search
Training accuracy for tasks: [0.99881566, 0.9970701, 0.9990971, 0.99389833, 0.99222434]
Validation accuracy for tasks: [0.9962175, 0.9973319, 0.9979859, 0.9939486, 0.99216455]
Precision: [0.99626863, 0.9969397, 0.9980159, 0.9939516, 0.9921875]
Recall: [0.99626863, 0.9969397, 0.9980159, 0.9939516, 0.9916992]
F1 score: [0.99626863, 0.9969397, 0.9980159, 0.9939516, 0.9919395]
Nodes in hidden layer:  [(28, 32), (28, 64), (28, 110), (28, 156), (28, 202)]
Learning rates:  [0.0035332382550846446, 0.0035332382550846446, 0.002779625851893939, 0.002779625851893939, 0.002779625851893939]
------------------------


In [117]:
from tensorflow.keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [118]:
# later...
 
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
optimizer = Adam(clipvalue=0.5)
loaded_model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy',f1_m,precision_m, recall_m])
loaded_model.evaluate(np.reshape(task_list[task_id][4],[-1, image_size, image_size]), task_list[task_id][5], verbose=0)

Loaded model from disk


[0.025618267930628155, 0.99216455, 0.9919395, 0.9921875, 0.9916992]

In [156]:
task_id=4
current_stage_weights = Model_Perf_save['weights'][-1].copy()
select_task_shape = Model_Perf_save['shape'][task_id]
for j,weight_size in enumerate(select_task_shape):
    if len(weight_size) == 2:
        current_stage_weights[j][weight_size[0]:,weight_size[1]:]=0
    else:
        current_stage_weights[j][weight_size[0]:]=0
loaded_model.set_weights(current_stage_weights)
loaded_model.evaluate(np.reshape(task_list[task_id][4],[-1, image_size, image_size]), task_list[task_id][5], verbose=0)

[2.43884821596622, 0.021057786, 0.0, 0.0, 0.0]

In [143]:
loaded_model.set_weights(Model_Perf_save['weights'][-1])
loaded_model.evaluate(np.reshape(task_list[task_id][4],[-1, image_size, image_size]), task_list[task_id][5], verbose=0)

[2.43884821596622, 0.021057786, 0.0, 0.0, 0.0]

In [159]:
for i,j in enumerate(loaded_model.get_weights()):
    comparision_result = j == Model_Perf_save['weights'][-1][i]
    print(i, np.where(comparision_result == False))

0 (array([], dtype=int64), array([], dtype=int64))
1 (array([], dtype=int64), array([], dtype=int64))
2 (array([], dtype=int64),)
3 (array([], dtype=int64), array([], dtype=int64))
4 (array([], dtype=int64),)
